In [2]:
#process.py
from scipy.spatial import distance as dist
from imutils import face_utils
import numpy as np
import math
import imutils
import time
import dlib
import cv2
from cv2 import VideoWriter_fourcc, VideoWriter
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
from keras.models import load_model

In [3]:
def eye_brow_distance(leye,reye):
    global points
    distance = dist.euclidean(leye,reye)
    #calculating euclidean distance between left eye and right eye
    points.append(int(distance))
    #adding distance to a list
    return distance

In [6]:
def emotion_finder(faces,frame):
    global emotion_classifier
    emotions_list = ["angry" ,"disgust","fear", "happy", "sad", "surprise","neutral"]
    x,y,w,h = face_utils.rect_to_bb(faces)
    frame = frame[y:y+h,x:x+w]
    #region of interest - roi
    roi = cv2.resize(frame,(64,64))
    roi = roi.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi,axis=0)
    preds = emotion_classifier.predict(roi)[0]
    emotion_probability = np.max(preds)
    label = emotions_list[preds.argmax()]
    if label in ['fear','sad', 'neutral']:
        label = 'stressed'
    else:
        label = 'not stressed'
    return label

In [8]:
def normalize_values(points,disp):
    normalized_value = abs(disp - np.min(points))/abs(np.max(points) - np.min(points))
    #max-min normalization
    '''The distances between left and right eyebrows is normalized for each frame in the video and the stress value 
    is calculated using that normalized value '''
    stress_value = np.exp(-(normalized_value))
    return stress_value

In [9]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
emotion_classifier = load_model("_mini_XCEPTION.102-0.66.hdf5", compile=False)
print(emotion_classifier, flush = True)
input_video = cv2.VideoCapture('input_video.mp4')
'''input_video = cv2.VideoCapture(0)
fps = 30   #Frames per second
size1= (int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))) 
#size --- this will capture the width and height of the video
videoWriter = cv2.VideoWriter('MyVideo.avi', cv2.VideoWriter_fourcc('I','4','2','0'), fps, size1)

success, frame = input_video.read()
# read is a function not attribute since it is followed by ()
# read fxn reads frame by frame
# success is a boolean type variable

numFramesRemaining = 10*fps - 1 
# numFramesRemaining is the number of seconds video will get recorded

while success and numFramesRemaining>0:
    videoWriter.write(frame)
    success, frame = input_video.read()
    numFramesRemaining -= 1'''
points = []
stress_list = []
stressval_list = []
stressgraph = []
size=0
while(True):
    _,frame = input_video.read()
    if(not _): break
    frame = cv2.flip(frame,1)
    frame = imutils.resize(frame, width=500,height=500)
    
    (lBegin, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]
    (rBegin, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]

    #preprocessing the image
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    detections = detector(gray,0)
    for detection in detections:
        emotion = emotion_finder(detection,gray)
        cv2.putText(frame, emotion, (10,10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        shape = predictor(frame,detection)
        shape = face_utils.shape_to_np(shape)
        
        leyebrow = shape[lBegin:lEnd]
        reyebrow = shape[rBegin:rEnd]
            
        reyebrowhull = cv2.convexHull(reyebrow)
        leyebrowhull = cv2.convexHull(leyebrow)

        cv2.drawContours(frame, [reyebrowhull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [leyebrowhull], -1, (0, 255, 0), 1)

        distance = eye_brow_distance(leyebrow[-1],reyebrow[0])
        stress_value = normalize_values(points,distance)
        print(stress_value)
        #if stress_value!=1.0: stress_list.append(stress_list)
        if math.isnan(stress_value):
            continue
        cv2.putText(frame,"stress level:{}".format(str(int(stress_value*100))),(20,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        stress_list.append(frame)

    height, width, layers = frame.shape
    size = (width,height)
    stressval_list.append(stress_value)
out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
input_video.release()
print("END REACHED")
for i in range(len(stress_list)):
    out.write(stress_list[i])


C:\Users\hp\AppData\Local\Temp/ipykernel_4692/1942540570.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  normalized_value = abs(disp - np.min(points))/abs(np.max(points) - np.min(points))


0.0
0.9334352252850698
0.3635805007669541
0.3635805007669541
0.3582875988499648
0.44392036003607943
0.3582875988499648
0.43726467206980746
0.3600942930281762
0.36700475791317283
0.42513462792741236
0.42513462792741236
0.600251518586586
0.4179413128445052
0.600251518586586
0.600251518586586
0.600251518586586
0.3654039487624784
0.36725866099476145
0.46900725773563534
0.6019572715378986
0.4161390680694228
0.6019572715378986
0.4104102675478394
0.5264957384941846
0.5264957384941846
0.5264957384941846
0.5884916537362881
0.4104102675478394
0.5313434908333113
0.46900725773563534
0.6753395551030256
0.46900725773563534
0.5264957384941846
0.46900725773563534
0.5963005971398023
0.5264957384941846
0.6753395551030256
0.5884916537362881
0.6019572715378986
0.6753395551030256
0.6019572715378986
0.7725518584743519
0.7648290948039481
0.86614464490287
0.9808409782110764
1.0
0.48216231687059996
0.9642616587435098
0.48216231687059996
0.48216231687059996
0.49898400873493554
0.46723777738325506
0.433365635930

# Training the model

In [6]:
#train.py
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

num_classes = 7 #number of labels
img_rows,img_cols = 48,48
batch_size = 32 #number of training examples utilized in one iteration

train_data_dir = 'train' 
validation_data_dir = 'test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='grayscale',
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)


model = Sequential()


# LAYER 1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


# LAYER 2

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


# LAYER 3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


# LAYER 4

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


# LAYER 5

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


# LAYER 6

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 

In [7]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Users\hp\BIOMETRICS\Stress_Detector\Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3, #epoch 1, 2, 3 if not improved, it will stop running
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history = model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

model.save('trained_model.hdf5')

D:\softwares__\Anaconda\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\hp\AppData\Local\Temp/ipykernel_12520/2862243727.py:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
755/755 [==============================] - ETA: 0s - loss: 2.1704 - accuracy: 0.1925
Epoch 00001: val_loss improved from inf to 1.78362, saving model to Users\hp\BIOMETRICS\Stress_Detector\Emotion_little_vgg.h5
755/755 [==============================] - 359s 468ms/step - loss: 2.1704 - accuracy: 0.1925 - val_loss: 1.7836 - val_accuracy: 0.2594 - lr: 0.0010
Epoch 2/25
755/755 [==============================] - ETA: 0s - loss: 1.8319 - accuracy: 0.2306
Epoch 00002: val_loss did not improve from 1.78362
755/755 [==============================] - 354s 468ms/step - loss: 1.8319 - accuracy: 0.2306 - val_loss: 1.7851 - val_accuracy: 0.2571 - lr: 0.0010
Epoch 3/25
755/755 [==============================] - ETA: 0s - loss: 1.7986 - accuracy: 0.2545
Epoch 00003: val_loss improved from 1.78362 to 1.78306, saving model to Users\hp\BIOMETRICS\Stress_Detector\Emotion_little_vgg.h5
755/755 [==============================] - 336s 445ms/step - loss: 1.7986 - accuracy: 0.2545 - val_loss: 1.78

In [6]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Accuracy and Loss Graph', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

NameError: name 'history' is not defined

# Testing the model

In [7]:
#testing
#test.py
from keras.preprocessing.image import img_to_array
import cv2
from keras.models import load_model
import numpy as np
# loading files
haar_file="haarcascade_frontalface_default (1).xml"
emotion_model='_mini_XCEPTION.102-0.66.hdf5'

cascade=cv2.CascadeClassifier(haar_file)
emotion_classifier=load_model(emotion_model,compile=True)
emotion_names=["angry","disgust","fear", "happy", "sad", "surprise","neutral"]
frame=cv2.imread('yy.jpeg')
gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
faces=cascade.detectMultiScale(gray_frame,1.5,5)
text=[]
for (x,y,w,h) in faces:
    roi=gray_frame[y:y+h,x:x+w]
    roi=cv2.resize(roi,(64,64))
    roi=roi.astype("float")/255.0
    roi=img_to_array(roi)
    roi=np.expand_dims(roi,axis=0)
    
    predicted_emotion=emotion_classifier.predict(roi)[0]
    probab=np.max(predicted_emotion)
    label=emotion_names[predicted_emotion.argmax()]
    percen=predicted_emotion*100
    for j in range(7):
        text.append(emotion_names[j]+" : "+str(percen[j]))
    for i in range(7):    
        #cv2.putText(frame,text[i],(5,i*30+15),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
        print(text[i])
    cv2.putText(frame,label,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
cv2.imwrite('yy__result.jpeg', frame)

angry : 6.643951
disgust : 0.619694
fear : 7.966957
happy : 53.667473
sad : 4.416847
surprise : 1.6760803
neutral : 25.008991


True

In [25]:
print("You are ", label)
tips = {"fear":["Drink Water","Get a good night's sleep","Eat wholesome meals","Go for a walk","Turn off news feed/social media","Talk to someone"],
       "angry":["Repeat gentle phrases to yourself","Take a walk","Use virtualization to calm down","Focus on your breathing","Phone a friend","Watch a standup comedy"],
       "sad":["Do things you enjoy (or used to)","Get quality exercise","Challenge negative thinking","Get a nutritious diet","List what you like about yourself"]}


website_links = {"fear":["https://www.businessinsider.in/science/health/heres-how-to-take-care-of-yourself-if-youre-feeling-scared-or-sad-right-now/articleshow/55342883.cms",
                "https://mhanational.org/what-can-i-do-when-im-afraid"],
        "angry":["https://www.thehotline.org/resources/how-to-cool-off-when-youre-angry/",
                "https://www.verywellmind.com/5-things-to-do-if-you-feel-angry-5092021"],
        "sad":["https://www.vandrevalafoundation.com/","https://www.healthline.com/health/mental-health/why-am-i-sad-for-no-reason"]}


youtube_links = {"fear":["https://www.youtube.com/watch?v=IAODG6KaNBc"],
                "angry":["https://www.youtube.com/watch?v=P6aPg3YBvBQ"],
                "sad":["https://www.youtube.com/watch?v=P6aPg3YBvBQ"]}


song_links = {"fear":[],
             "angry":[],
             "sad":[]}


if (label=='happy' or label=='neutral' or label=='surprised'):
    #songs
    print("hi")
    
elif (label=='angry' or label=='sad'):
    print(tips.get(label))
    for i in website_links.get(label):
        print(i)
    for j in youtube_links.get(label):
        print(j)
    #songs
    #tips
    #resources
    
elif (label=='fear'):
    #kmkm
    print(tips.get('fear'))
    print(website_links.get('fear'))
    print(youtube_links.get('fear'))

You are  sad
['Do things you enjoy (or used to)', 'Get quality exercise', 'Challenge negative thinking', 'Get a nutritious diet', 'List what you like about yourself']
https://www.vandrevalafoundation.com/
https://www.healthline.com/health/mental-health/why-am-i-sad-for-no-reason
https://www.youtube.com/watch?v=P6aPg3YBvBQ


In [4]:
print(label)

sad


In [22]:
import webbrowser
webbrowser.open('https://www.vandrevalafoundation.com/')

True

In [ ]:
for i in website_links.get('sad'):
    print(i)

In [3]:
from playsound import playsound

labels = ["happy", "angry", "fear", "disgust", "sad", "surprised", "neutral"]
label = "happy"
tips = {"fear":["Drink water","Get a good night's sleep","Eat wholesome meals","Go for a walk","Turn off news feed/social media","Talk to someone"],
        "angry":["Repeat gentle phrases to yourself","Take a walk","Use visualization to calm down","Focus on your breathing","Phone a friend","Watch a stand up comedy"],
        "sad":["Do things you enjoy (or used to)","Get quality exercise","Eat a nutritious diet","Challenge negative thinking"]
       }
website_links = {"fear":["https://www.businessinsider.in/science/health/heres-how-to-take-care-of-yourself-if-youre-feeling-scared-or-sad-right-now/articleshow/55342883.cms","https://mhanational.org/what-can-i-do-when-im-afraid"],
                 "angry":["https://www.thehotline.org/resources/how-to-cool-off-when-youre-angry/","https://www.mayoclinic.org/healthy-lifestyle/adult-health/in-depth/anger-management/art-20045434"],
                 "sad":["https://www.vandrevalafoundation.com/","https://www.healthline.com/health/depression/recognizing-symptoms#fatigue"]
                }
youtube_links = {"fear":["https://www.youtube.com/watch?v=IAODG6KaNBc"],
                 "angry":["https://www.youtube.com/watch?v=P6aPg3YBvBQ"],
                 "sad":["https://www.youtube.com/watch?v=P6aPg3YBvBQ"]
                }
song_links = {"fear":["https://www.youtube.com/watch?v=GyA8ccqwp-4&feature=youtu.be","https://www.bing.com/videos/search?q=alone+part+2&docid=607990227673701963&mid=1B6860319511BF2C5CC21B6860319511BF2C5CC2&view=detail&FORM=VIRE"],
              "angry":["https://www.youtube.com/watch?v=e74wLJ_KRes&feature=youtu.be","https://www.youtube.com/watch?v=JwWz-94a_Hk&feature=youtu.be"],
              "sad":["https://www.youtube.com/watch?v=25ROFXjoaAU&feature=youtu.be","https://www.youtube.com/watch?v=BzE1mX4Px0I"],
              "happy":["https://www.youtube.com/watch?v=vGZhMIXH62M","https://www.youtube.com/watch?v=Pkh8UtuejGw"]
             }
tunes = {"fear":'fear.mp3',
         "angry":'angry.mp3',
         "sad":'sad.mp3'
         }

if (label == "happy"):
    # songs
    print("Here are some song suggestions for your mood:")
    for s in song_links.get('happy'):
        print(s)

elif (label == "angry"):
    # songs
    print("Here are some song suggestions for your mood:")
    for s in song_links.get('angry'):
        print(s)
    # tips
    print("Here are some tips to help you feel better:")
    for i in tips.get('angry'):
        print("-> "+i)
    # resources
    print("Here are some resources that you may find beneficial:")
    for j in website_links.get('angry'):
        print(j)
    for k in youtube_links.get('angry'):
        print(k)
    # tunes
    print("Here's a tune that will help you calm down.")
    playsound(tunes.get('angry'))

elif (label == "fear"):
    # songs
    print("Here are some song suggestions for your mood:")
    for s in song_links.get('fear'):
        print(s)
    # tips
    print("Here are some tips to help you feel better:")
    for i in tips.get('fear'):
        print("-> "+i)
    # resources
    print("Here are some resources that you may find beneficial:")
    for j in website_links.get('fear'):
        print(j)
    for k in youtube_links.get('fear'):
        print(k)
    # tunes
    print("Here's a tune that will make you feel better.")
    playsound(tunes.get('fear'))

elif (label == "sad"):
    # songs
    print("Here are some song suggestions for your mood:")
    for s in song_links.get('sad'):
        print(s)
    # tips
    print("Here are some tips to help you feel better:")
    for i in tips.get('sad'):
        print("-> "+i)
    # resources
    print("Here are some resources that you may find beneficial:")
    for j in website_links.get('sad'):
        print(j)
    for k in youtube_links.get('sad'):
        print(k)
    # tunes
    print("Listen to a tune that will soothe you.")
    playsound(tunes.get('sad'))


Here are some song suggestions for your mood:
https://www.youtube.com/watch?v=vGZhMIXH62M
https://www.youtube.com/watch?v=Pkh8UtuejGw
